In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import pandas as pd
import numpy as np

import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data.sampler import RandomSampler

"""Change to the data folder"""
new_path = "./new_train/new_train"
val_path = "./new_val_in/new_val_in" 
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [2]:
torch.cuda.is_available()

True

### Create a dataset class 

In [3]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [4]:
batch_sz = 128

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    out = torch.LongTensor(out)
    inp = torch.tensor(inp, dtype=torch.float)
    out = torch.tensor(out, dtype=torch.float)
    return [inp, out]

# sampler = RandomSampler(train_dataset, num_samples = 1000, replacement=True)
sampler = RandomSampler(train_dataset)

train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0, sampler = sampler, drop_last=True)


In [5]:
class NN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(NN, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)

        hidden = self.init_hidden(batch_size)
        out, hidden = self.gru(x, hidden)

#         out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden.to(device)

In [6]:
from tqdm import tqdm_notebook as tqdm
def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    running_loss = 0
    for batch_idx, (data, target) in enumerate(iterator):
        data, target = torch.reshape(data, (batch_sz, 60, -1)).to(device), torch.reshape(target[:,:,:,:2], (batch_sz, 60, -1)).to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.MSELoss()(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        iterator.set_postfix(loss=running_loss)
    return running_loss
#         iterator.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))

In [7]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, agent in test_loader:
            data, agent = torch.reshape(data, (4, 60, -1)).to(device)
            output = model(data)

In [8]:
# input dimension
input_dim = 76
hidden_dim = 60  # hidden layer dimension
layer_dim = 1   # number of hidden layers
output_dim = 60   # output dimension

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"  
    
learning_rate = 0.0001
momentum = 0.5

In [95]:
net = NN(input_dim, output_dim, hidden_dim, layer_dim)
model = net.to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum, weight_decay=1e-5)
num_epoch = 10
losses = []
for epoch in range(1, num_epoch + 1):
    loss = train(model, device, train_loader, optimizer, epoch)
    losses.append(loss)
    torch.save(model.state_dict(), 'checkpoints/train-epoch-lstm{}.pth'.format(epoch + 1)) 
    
with open("losses.txt", "w") as output:
    output.write(str(losses))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


KeyboardInterrupt: 

## Test and predict

In [9]:
model=torch.load("checkpoints/train-epoch-lstm11.pth",map_location=device)

In [10]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

In [13]:
batch_sz = 4

def my_test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
#     print(batch)
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = torch.tensor(inp, dtype=torch.float)
    agent = [scene['agent_id'] for scene in batch]
    track = [scene['track_id'] for scene in batch]
    track_ids = []
    for scene in range(len(agent)):
        for track_id in range(len(track[scene])):
            if agent[scene] == track[scene][track_id][0][0]:
                track_ids.append(track_id)
                break
    return [inp, track_ids]

# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=val_path)
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_test_collate, num_workers=0, drop_last=True)


In [14]:
def predict(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    predictions = []
    with torch.no_grad():
        for data, agent in test_loader:
            data = torch.reshape(data, (4, 60, -1)).to(device)
            output = torch.reshape(model(data), (4, 60, 30, -1))
            for i in range(len(agent)):
                scene = output[i][agent[i]]
                predictions.append(scene)
    predict = [torch.reshape(t, (-1,)) for t in predictions]
    sample = pd.read_csv('sample_submission.csv')
    preds_df = sample.set_index('ID')
    for i in range(3200):
        preds_df.iloc[i] = predict[i].tolist()
    return preds_df

In [16]:
net = NN(input_dim, output_dim, hidden_dim, layer_dim)
net.load_state_dict(torch.load("checkpoints/train-epoch-lstm11.pth",map_location=device))
model = net.to(device)
preds = predict(model, device, val_loader)

In [17]:
preds.to_csv('test_preds_lstm.csv')

In [18]:
preds.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
10002,580.965454,622.587402,580.506714,623.019836,580.574707,623.074463,581.070801,623.213379,580.779358,622.993713,...,580.882629,622.560059,580.616394,623.743835,580.856018,623.11084,580.314697,622.305115,580.905457,622.043762
10015,462.188019,908.581909,461.594116,909.208740,461.667908,909.088013,461.623138,909.446472,462.104492,908.725037,...,461.833008,908.804932,461.777344,908.774353,462.054871,909.23230,462.124878,908.561279,461.674530,908.385803
10019,462.188019,908.581909,461.594116,909.208740,461.667908,909.088013,461.623138,909.446472,462.104492,908.725037,...,461.833008,908.804932,461.777344,908.774353,462.054871,909.23230,462.124878,908.561279,461.674530,908.385803
10028,567.789978,603.547302,567.364868,603.888855,567.497314,603.890381,567.860107,604.023132,567.574158,603.944153,...,567.785461,603.393066,567.472839,604.580383,567.793335,603.94812,567.230957,603.265930,567.765320,602.980408
1003,642.411682,712.034607,642.380066,712.742920,642.052734,712.704285,642.781128,712.907715,642.394165,712.321533,...,642.573120,712.379456,642.190857,713.039124,642.393616,712.75293,642.178223,711.937927,642.639954,711.609375
